In [1]:
import sqlite3
import pandas as pd
import os

# Create table

In [2]:
# Function to create database schema
def create_database():
    with sqlite3.connect('university_admissions.db') as conn:
        cursor = conn.cursor()

        # Create table: universities
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS universities (
            code TEXT PRIMARY KEY,
            short_name TEXT,
            full_name TEXT
        )
        ''')

        # Create table: admission_methods
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS admission_methods (
            id INTEGER PRIMARY KEY,
            name TEXT
        )
        ''')

        # Create table: admission_scores
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS admission_scores (
            id INTEGER PRIMARY KEY,
            university_code TEXT NOT NULL,
            major_code TEXT NOT NULL,
            major_name TEXT NOT NULL,
            admission_method_id INTEGER NOT NULL,
            year INTEGER NOT NULL,
            subject_combination TEXT,
            score REAL,
            note TEXT,
            FOREIGN KEY (university_code) REFERENCES universities (code),
            FOREIGN KEY (admission_method_id) REFERENCES admission_methods (id)
        )
        ''')

        conn.commit()


In [3]:
create_database()

# Insert data

In [4]:
# Import data from Excel files
file_path_universities = r'E:\LLM_clone\Tdtu-chatbot\notebooks\DataSQL\Danh_sach_truong_dai_hoc.xlsx'
data_universities = pd.read_excel(file_path_universities)

file_path_admission_methods = r'E:\LLM_clone\Tdtu-chatbot\notebooks\DataSQL\admission_methods.xlsx'
data_admission_methods = pd.read_excel(file_path_admission_methods)

with sqlite3.connect('university_admissions.db') as conn:
    cursor = conn.cursor()

    # Insert data into universities table
    for index, row in data_universities.iterrows():
        cursor.execute('''
        INSERT OR IGNORE INTO universities (code, short_name, full_name)
        VALUES (?, ?, ?)
        ''', (row['Mã Trường'], row['Viết Tắt'], row['Tên Trường']))

    # Insert data into admission_methods table
    for index, row in data_admission_methods.iterrows():
        cursor.execute('''
        INSERT OR IGNORE INTO admission_methods (id, name)
        VALUES (?, ?)
        ''', (row['id'], row['name']))

    conn.commit()


In [5]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    for row in rows:
        print(row)
    conn.close()
    

In [6]:
read_sql_query('SELECT * FROM universities LIMIT 20;',
               "university_admissions.db")

('QSB', 'BKU', 'Trường Đại học Bách Khoa TP. Hồ Chí Minh (Ho Chi Minh City University of Technology)')
('SPK', 'HCMUTE', 'Trường Đại học Sư phạm Kỹ thuật TP. Hồ Chí Minh (Ho Chi Minh City University of Technology and Education)')
('FPT', 'FPT', 'Trường Đại học FPT (FPT University)')
('NTS', 'FTU2', 'Trường Đại học Ngoại thương - Cơ sở 2 TP. Hồ Chí Minh (Foreign Trade University, Campus 2)')
('QST', 'HCMUS', 'Trường Đại học Khoa học Tự nhiên TP. Hồ Chí Minh (University of Science, Ho Chi Minh City)')
('TTD', 'TDTU', 'Trường Đại học Tôn Đức Thắng (Ton Duc Thang University)')
('KSA', 'UEH', 'Trường Đại học Kinh tế TP. Hồ Chí Minh (University of Economics Ho Chi Minh City)')
('DMS', 'UFM', 'Trường Đại học Tài chính - Marketing (University of Finance and Marketing)')
('QSC', 'UIT', 'Trường Đại học Công nghệ Thông tin TP. Hồ Chí Minh (University of Information Technology, Ho Chi Minh City)')
('DVL', 'VLU', 'Trường Đại học Văn Lang (Van Lang University)')
('YDS', 'UMP', 'Trường Đại học Y Dược

In [7]:
# Corrected SQL query
query = 'SELECT * FROM universities WHERE code = "QSB" LIMIT 20;'

# Connect to the database and execute the query
with sqlite3.connect("university_admissions.db") as conn:
    df = pd.read_sql_query(query, conn)

# Display the result
print(df)


  code short_name                                          full_name
0  QSB        BKU  Trường Đại học Bách Khoa TP. Hồ Chí Minh (Ho C...


In [8]:
read_sql_query('SELECT * FROM admission_methods LIMIT 20;',
               "university_admissions.db")

(1, 'Thi THPT')
(2, 'Học bạ - Kết quả học tập THPT đợt 1')
(3, 'Học bạ - Kết quả học tập THPT đợt 2')
(4, 'DGNL')
(5, 'UTXT đợt 1')
(6, 'UTXT đợt 2')
(7, 'Xét riêng')


In [13]:
# Function to insert admission scores from folder structure
def insert_admission_scores_from_folders(base_path):
    """
    Insert admission scores from Excel files stored in a folder structure by university and year.

    :param base_path: The base directory containing university folders with year subfolders.
    """
    with sqlite3.connect('university_admissions.db') as conn:
        cursor = conn.cursor()

        # Traverse through each university folder
        for university_folder in os.listdir(base_path):
            university_path = os.path.join(base_path, university_folder)

            if os.path.isdir(university_path):  # Ensure it's a directory
                tables_path = os.path.join(university_path, "Tables")

                if os.path.isdir(tables_path):  # Ensure 'Tables' directory exists
                    # Traverse through each year folder within the 'Tables' directory
                    for year_folder in os.listdir(tables_path):
                        year_path = os.path.join(tables_path, year_folder)

                        if os.path.isdir(year_path):  # Ensure it's a directory
                            try:
                                year = int(year_folder)  # Parse year from folder name
                            except ValueError:
                                continue  # Skip if the folder name is not a year

                            # Traverse through each Excel file in the year folder
                            for file_name in os.listdir(year_path):
                                if file_name.endswith('.xlsx'):  # Only process Excel files
                                    file_path = os.path.join(year_path, file_name)
                                    admission_method = int(file_name[0])
                                    # Read the Excel file
                                    data = pd.read_excel(file_path)

                                    # Insert data into admission_scores table
                                    for index, row in data.iterrows():
                                        cursor.execute('''
                                        INSERT OR IGNORE INTO admission_scores (
                                            id, university_code, major_code, major_name, admission_method_id,
                                            year, subject_combination, score, note
                                        )
                                        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                                        ''', (
                                            None,  # Auto-increment ID
                                            university_folder,  # University code from folder name
                                            row['Mã ngành'],
                                            row['Tên ngành'],
                                            admission_method,  # Default admission method ID (e.g., Thi THPT)
                                            year,
                                            row['Tổ hợp'],
                                            row['Điểm chuẩn'],
                                            row['Ghi chú']
                                        ))

        conn.commit()


In [14]:
insert_admission_scores_from_folders(r'E:\LLM_clone\Tdtu-chatbot\notebooks\Data Tuyển sinh')

In [15]:
# Corrected SQL query
query = 'SELECT * FROM universities WHERE code = "QSB" '

# Connect to the database and execute the query
with sqlite3.connect("university_admissions.db") as conn:
    df = pd.read_sql_query(query, conn)

# Display the result
print(df)


  code short_name                                          full_name
0  QSB        BKU  Trường Đại học Bách Khoa TP. Hồ Chí Minh (Ho C...


In [21]:
# Corrected SQL query
query = 'SELECT * FROM admission_scores;'

# Connect to the database and execute the query
with sqlite3.connect("university_admissions.db") as conn:
    df = pd.read_sql_query(query, conn)

# Display the result
df[:5]

,id,university_code,major_code,major_name,admission_method_id,year,subject_combination,score,note
0,1,TDTU,7220201,Ngôn ngữ Anh,1,2021,D01; D11,"35,60","Môn nhân hệ số 2, điều kiện: Anh\n Điểm trúng ..."
1,2,TDTU,7310630,Việt Nam học (Chuyên ngành: Du lịch và lữ hành),1,2021,A01; C00; C01; D01,"33,30","Môn nhân hệ số 2, điều kiện: A01, D01: Anh C00..."
2,3,TDTU,7310630Q,Việt Nam học (Chuyên ngành: Du lịch và quản lý...,1,2021,A01; C00; C01; D01,"34,20","Môn nhân hệ số 2, điều kiện: A01, D01: Anh C00..."
3,4,TDTU,7340101,Quản trị kinh doanh (Chuyên ngành: Quản trị ng...,1,2021,A00; A01; D01,"36,00","Môn nhân hệ số 2, điều kiện: A00: Toán, A01, D..."
4,5,TDTU,7340115,Marketing,1,2021,A00; A01; D01,"36,90","Môn nhân hệ số 2, điều kiện: A00: Toán, A01, D..."


# View db

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1835 entries, 0 to 1834
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1835 non-null   int64 
 1   university_code      1835 non-null   object
 2   major_code           1835 non-null   object
 3   major_name           1835 non-null   object
 4   admission_method_id  1835 non-null   int64 
 5   year                 1835 non-null   int64 
 6   subject_combination  690 non-null    object
 7   score                1834 non-null   object
 8   note                 806 non-null    object
dtypes: int64(3), object(6)
memory usage: 129.2+ KB


In [23]:
df.isnull().sum()

id                        0
university_code           0
major_code                0
major_name                0
admission_method_id       0
year                      0
subject_combination    1145
score                     1
note                   1029
dtype: int64

In [24]:
print(df['year'].unique())
print(df['admission_method_id'].unique())

[2021 2022 2023 2024]
[1 2 3 5 6 4]


In [27]:
# Corrected SQL query
query = 'SELECT * FROM admission_scores WHERE score IS NULL;'

# Connect to the database and execute the query
with sqlite3.connect("university_admissions.db") as conn:
    df = pd.read_sql_query(query, conn)

# Display the result
df

,id,university_code,major_code,major_name,admission_method_id,year,subject_combination,score,note
0,763,TDTU,7580302,Quản lý xây dựng,3,2023,None,None,None
